# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель.

Практическая работа разделена на 2 части:
1. Построение нграмой языковой модели - основная часть, 10 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 6 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

In [143]:
import numpy as np
from collections import defaultdict
from typing import List, Dict, Tuple

# 1. Построение нграмной языковой модели. (10 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле:
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$

Поскольку униграмы не содержат в себе какого-дибо контекста, вероятность униграмы можно посчитать поделив кол-во этой слова на общее количество слов в обучающей выборке.


In [146]:
# в первую очередь нам понадобится подсчитать статистику по обучающей выборке
def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
    ngrams = defaultdict(int)
    # TODO реализуйте функцию, которая подсчитывает все 1-gram 2-gram ... order-gram ngram'ы в тексте
    for sentence in train_text:
      tokens = sentence.split()
      # добавляем символ начала предложения (<s>) в начало, если bos=True
      if bos:
        tokens = ['<s>'] + tokens
      # добавляем символ конца предложения (</s>) в конец, если eos=True
      if eos:
        tokens = tokens + ['</s>']
      for i in range(1, order+1):
        for j in range(len(tokens) - i + 1):
                ngrams[tuple(tokens[j:j+i])] += 1


    #
    return dict(ngrams)

In [147]:
def test_count_ngrams():
    assert count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True) == {
        ('<s>',): 1,
        ('привет',): 2,
        ('как',): 1,
        ('дела',): 1,
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=True) == {
        ('привет',): 2,
        ('как',): 1,
        ('дела',): 1,
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=False) == {
        ('привет',): 2,
        ('как',): 1,
        ('дела',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=2, bos=False, eos=False) == {
        ('привет',): 2,
        ('как',): 1,
        ('дела',): 1,
        ('привет', 'привет'): 1,
        ('привет', 'как'): 1,
        ('как', 'дела'): 1
    }
    assert count_ngrams(['привет ' * 6], order=2, bos=False, eos=False) == {
        ('привет',): 6,
        ('привет', 'привет'): 5
    }
    result = count_ngrams(['практическое сентября',
                           'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=5)
    assert result[('<s>',)] == 3
    assert result[('32',)] == 3
    assert result[('<s>', 'в', 'офлайне', 'в', '32')] == 1
    assert result[('офлайне', 'в', '32', '12', '</s>')] == 1
    print('Test 1a passed')


test_count_ngrams()

Test 1a passed



Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю.

Чтобы избежать данного недостатка, вводится специальное сглаживание - add-k сглаживание ([Additive, Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.


In [148]:
# функция подсчета вероятности через количество со сглаживанием Лапласа
def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
    # подсчитывет ngram со сглаживанием Лапласа
    # TODO
    # если размер словаря не указан, рассчитываем его
    if V is None:
        V = sum(1 for key in counts if len(key) == 1)
    # считаем количество предшественников n-граммы.
    prefix_count = sum(value for key, value in counts.items()
                       if len(key) == len(ngram) and key[:-1] == ngram[:-1])
    # получаем частоту n-граммы из словаря
    count = counts.get(ngram, 0)
     # вычисляем вероятность n-граммы по формуле сглаживания Лапласа
    prob = (count + k) / (prefix_count + k * V)

    return prob

In [149]:
def test_calculate_ngram_prob():
    counts = count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)
    assert calculate_ngram_prob(('в', 'офлайне'), counts) == 0.5
    assert calculate_ngram_prob(('в', ), counts) == 4/25
    assert calculate_ngram_prob(('в', ), counts, k=0.5) == (4+0.5)/(25+0.5*12)
    assert calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts) == 1.0
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=1) == 0.1875
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=0) == 0.0
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=1) == 0.0625
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

    print("Test 1.b passed")


test_calculate_ngram_prob()

Test 1.b passed


Основной метрикой язковых моделей является перплексия.

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$


In [170]:
# Языковая модель
class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = eos
        self.bos = bos
        self.k = k
        self.vocab = predefined_vocab
        self.ngrams_count = None

    @property
    def V(self) -> int:
        return len(self.vocab)
    #добавляем уже готовые функции из заданий выше
    def _count_ngrams(self, train_text: List[str]) -> Dict[Tuple[str], int]:
      ngrams = defaultdict(int)
      for sentence in train_text:
            tokens = sentence.split()
            if self.bos:
                tokens = ['<s>'] + tokens
            if self.eos:
                tokens = tokens + ['</s>']

            for i in range(1, self.order + 1):
                for j in range(len(tokens) - i + 1):
                    ngrams[tuple(tokens[j:j+i])] += 1
      return dict(ngrams)

    def _calculate_ngram_prob(self, ngram: Tuple[str]) -> float:
      #if V is None:
          #V = sum(1 for key in self.ngrams_count if len(key) == 1)

          prefix_count = sum(value for key, value in self.ngrams_count.items()
                            if len(key) == len(ngram) and key[:-1] == ngram[:-1])

          count = self.ngrams_count.get(ngram, 0)
          prob = (count + self.k) / (prefix_count + self.k * self.V)

          return prob

    def fit(self, train_text: List[str]) -> None:
        # TODO
        # Подсчет vocab и ngrams_count по обучающей выборке
        self.ngrams_count = self._count_ngrams(train_text)
        self.vocab = {ngram[0] for ngram in self.ngrams_count.keys() if len(ngram) == 1}


    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        # TODO
        # считаем логарифм вероятности конкретной нграмы
        return np.log(self._calculate_ngram_prob(ngram))


    def predict_log_proba(self, words: List[str]) -> float:
        if self.bos:
            words = ['<s>'] + words
        if self.eos:
            words = words + ['</s>']
        logprob = 0
        # TODO
        # применяем chain rule, чтобы посчитать логарифм вероятности всей строки
        logprob = sum(self.predict_ngram_log_proba(tuple(words[max(0, i - self.order + 1):i + 1]))
                                                    for i in range(len(words)))

        return logprob

    def ppl(self, text: List[str]) -> float:
        #TODO
        # подсчет перплексии
        # Для того, чтобы ваш код был численно стабильным,
        #    не считайте формулу напрямую, а воспользуйтесь переходом к логарифмам вероятностей
        # сумма логарифмов вероятностей слов в тексте
        total_logprob = sum(self.predict_log_proba(line.split()) for line in text)
        # общее количество слов в тексте, включая начальные и конечные токены
        total_words = sum(len(line.split()) for line in text) + (self.bos + self.eos) * len(text)
        # вычисляем перплексию как экспоненту от отрицательного среднего логарифма вероятности
        # если общее количество слов равно 0, перплексия устанавливается в бесконечность
        return np.exp(-total_logprob / total_words) if total_words > 0 else float('inf')

In [171]:
def test_lm():
    train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
    global lm
    lm = NgramLM(order=2)
    lm.fit(train_data)
    assert lm.V == 14
    assert np.isclose(lm.predict_log_proba(['мы']), lm.predict_log_proba(["если"]))
    assert lm.predict_log_proba(["по-моему"]) > lm.predict_log_proba(["если"])

    gt = ((3+1)/(41 + 14) * 1/(3+14))**(-1/2)
    ppl = lm.ppl([''])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"

    gt = ((3+1)/(41 + 14) * 1/(3+14) * 1/(14)) ** (-1/3)
    ppl = lm.ppl(['ЧТО'])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"

    test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]
    ppl = lm.ppl(test_data)
    assert round(ppl, 2) == 7.33, f"{ppl}"
test_lm()

# 2. Предсказания с помощью языковой модели (6 балла)

In [172]:
def predict_next_word(lm: NgramLM, prefix: List[str], topk=4):
    # TODO реализуйте функцию, которая предсказывает продолжение фразы.
    # верните topk наиболее вероятных продолжений фразы prefix

    # Предсказание следующих слов на основе префикса
    top_prob = sorted(
        # получаем логарифмические вероятности для каждого слова из словаря после префикса
        ((word, lm.predict_ngram_log_proba(tuple(prefix + [word])))
         for word in lm.vocab),
        # сортируем по убыванию логарифмической вероятности
        key=lambda e: e[1],
        reverse=True
    )[:topk]

    return [(word, log_prob) for word, log_prob in top_prob]

Попробуйте обучить ngram языковую модель на нескольких стихотворениях. Не забудьте трансформировать стихотворение в удобный для ngram модели формат (как сделать так, чтобы модель моделировала рифму?).
Попробуйте сгенерировать продолжение для стихотворения с помощью такой языковой модели.

In [173]:
poem = [ """
До свиданья, друг мой, до свиданья.
Милый мой, ты у меня в груди.
Предназначенное расставанье
Обещает встречу впереди.
До свиданья, друг мой, без руки, без слова,
Не грусти и не печаль бровей, —
В этой жизни умирать не ново,
Но и жить, конечно, не новей.
"""]
lm = NgramLM(order=3)
lm.fit(poem)

predict_next_word(lm, ["мой"])

[('умирать', -3.6375861597263857),
 ('меня', -3.6375861597263857),
 ('мой,', -3.6375861597263857),
 ('жить,', -3.6375861597263857)]

In [196]:
train = """
У лукоморья дуб зелёный;
Златая цепь на дубе том:
И днём и ночью кот учёный
Всё ходит по цепи кругом;
Идёт направо — песнь заводит,
Налево — сказку говорит.
Там чудеса: там леший бродит,
Русалка на ветвях сидит;
Там на неведомых дорожках
Следы невиданных зверей;
Избушка там на курьих ножках
Стоит без окон, без дверей;
Там лес и дол видений полны;
Там о заре прихлынут волны
На брег песчаный и пустой,
И тридцать витязей прекрасных
Чредой из вод выходят ясных,
И с ними дядька их морской;
Там королевич мимоходом
Пленяет грозного царя;
Там в облаках перед народом
Через леса, через моря
Колдун несёт богатыря;
В темнице там царевна тужит,
А бурый волк ей верно служит;
Там ступа с Бабою Ягой
Идёт, бредёт сама собой,
Там царь Кащей над златом чахнет;
Там русский дух… там Русью пахнет!
И там я был, и мёд я пил;
У моря видел дуб зелёный;
Под ним сидел, и кот учёный
Свои мне сказки говорил.
"""

In [155]:
import re

In [197]:
normalized = train.strip().lower()
normalized = re.sub(r"\s+", " ", normalized)
normalized = re.sub(r"^\s*\$\$.+\$\$\s*$", "", normalized, flags=re.MULTILINE)
normalized = re.sub(r"\s+-\s+", " ", normalized)
normalized = re.sub(r"[^\w\d\.\?!\-\s]", " ", normalized)

In [198]:
sentences = re.split(r"[\.!?]+|\n+", normalized)
sentences = [sentence.strip() for sentence in sentences if sentence.strip() != ""]

In [199]:
data = sentences

lm = NgramLM(order=3)
lm.fit(data)

In [206]:
word = "дуб"
length = 6

# инициализация текста начальным словом
text = word
for _ in range(length):
    # получение последнего слова из текущего текста
    last_word = text.split()[-1]
    # предсказание следующего слова
    next_word = predict_next_word(lm, [last_word], topk=1)[0][0]
    # добавление предсказанного слова к тексту
    text += f" {next_word}"
text

'дуб зелёный златая цепь на курьих ножках'